In [ ]:
import pandas as pd
data_availability = {}

In [43]:
## Total available
import os
step = '1'

data_availability[step] = {"Description" : "Delete Missing data (Missing value from both channels)"}

dir = 'merged'
homes = [f for f in os.listdir(dir) if os.path.isdir(os.path.join(dir, f))]
for home in homes:
    files = ['Indoor.csv', 'Outdoor.csv']
    for file in files:
        file_path = os.path.join(dir, home, file)
        df = pd.read_csv(file_path, parse_dates=['BDDateTime'])
        total_sec = (df['BDDateTime'].max() - df['BDDateTime'].min()).total_seconds()
        expected_data = total_sec / 120
        available_data = df[~df['pm2_5_atm'].isna() | ~df['pm2_5_atm_b'].isna()].shape[0]
        availability = (available_data / expected_data) * 100 if expected_data > 0 else 0
        print(f"Home: {home}, File: {file}, Data Availability: {availability:.2f}%")
        data_availability[step][f'{home}_{file}'] = availability
        

Home: H1, File: Indoor.csv, Data Availability: 55.47%
Home: H1, File: Outdoor.csv, Data Availability: 45.67%
Home: H2, File: Indoor.csv, Data Availability: 91.92%
Home: H2, File: Outdoor.csv, Data Availability: 92.74%
Home: H3, File: Indoor.csv, Data Availability: 99.06%
Home: H3, File: Outdoor.csv, Data Availability: 90.15%
Home: H4, File: Indoor.csv, Data Availability: 88.16%
Home: H4, File: Outdoor.csv, Data Availability: 95.99%
Home: H5, File: Indoor.csv, Data Availability: 77.19%
Home: H5, File: Outdoor.csv, Data Availability: 56.40%
Home: H6, File: Indoor.csv, Data Availability: 97.48%
Home: H6, File: Outdoor.csv, Data Availability: 95.45%


In [44]:
## One channel observation
import os
step = '2'
data_availability[step] = {"Description" : "Delete observation with data from one channel"}

dir = 'merged'
homes = [f for f in os.listdir(dir) if os.path.isdir(os.path.join(dir, f))]
for home in homes:
    files = ['Indoor.csv', 'Outdoor.csv']
    for file in files:
        file_path = os.path.join(dir, home, file)
        df = pd.read_csv(file_path, parse_dates=['BDDateTime'])
        total_sec = (df['BDDateTime'].max() - df['BDDateTime'].min()).total_seconds()
        expected_data = total_sec / 120
        available_data = df[~df['pm2_5_atm'].isna() & ~df['pm2_5_atm_b'].isna()].shape[0]
        availability = (available_data / expected_data) * 100 if expected_data > 0 else 0
        print(f"Home: {home}, File: {file}, Data Availability: {availability:.2f}%")
        data_availability[step][f'{home}_{file}'] = availability
        

Home: H1, File: Indoor.csv, Data Availability: 55.47%
Home: H1, File: Outdoor.csv, Data Availability: 45.67%
Home: H2, File: Indoor.csv, Data Availability: 91.92%
Home: H2, File: Outdoor.csv, Data Availability: 92.74%
Home: H3, File: Indoor.csv, Data Availability: 99.06%
Home: H3, File: Outdoor.csv, Data Availability: 90.15%
Home: H4, File: Indoor.csv, Data Availability: 88.16%
Home: H4, File: Outdoor.csv, Data Availability: 95.99%
Home: H5, File: Indoor.csv, Data Availability: 77.19%
Home: H5, File: Outdoor.csv, Data Availability: 56.40%
Home: H6, File: Indoor.csv, Data Availability: 97.48%
Home: H6, File: Outdoor.csv, Data Availability: 95.45%


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298076 entries, 0 to 298075
Data columns (total 56 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   UTCDateTime          298076 non-null  object        
 1   mac_address          298076 non-null  object        
 2   firmware_ver         298076 non-null  float64       
 3   hardware             298076 non-null  object        
 4   current_temp_f       298076 non-null  float64       
 5   current_humidity     298076 non-null  float64       
 6   current_dewpoint_f   298075 non-null  float64       
 7   pressure             298073 non-null  float64       
 8   adc                  291770 non-null  float64       
 9   mem                  298073 non-null  float64       
 10  rssi                 298073 non-null  float64       
 11  uptime               298073 non-null  float64       
 12  pm1_0_cf_1           298073 non-null  float64       
 13  pm2_5_cf_1    

In [47]:
## Data with abnormal temperature and relative humidity
import os
step = '3'
data_availability[step] = {"Description" : "Delete observation with abnormal temperature and relative humidity (T<-200 or >1000 F, RH >100% or <0%)"}

dir = 'merged'
homes = [f for f in os.listdir(dir) if os.path.isdir(os.path.join(dir, f))]
for home in homes:
    files = ['Indoor.csv', 'Outdoor.csv']
    for file in files:
        file_path = os.path.join(dir, home, file)
        df = pd.read_csv(file_path, parse_dates=['BDDateTime'])
        total_sec = (df['BDDateTime'].max() - df['BDDateTime'].min()).total_seconds()
        expected_data = total_sec / 120

        #logics
        pm_logic = ~df['pm2_5_atm'].isna() & ~df['pm2_5_atm_b'].isna()
        rh_logic = (df['current_humidity'] < 100) & (df['current_humidity'] > 0)
        temp_logic = (df['current_temp_f'] > -200) & (df['current_temp_f'] < 1000)
        validity_logic = pm_logic & rh_logic & temp_logic

        #avialability check
        available_data = df[validity_logic].shape[0]
        availability = (available_data / expected_data) * 100 if expected_data > 0 else 0
        print(f"Home: {home}, File: {file}, Data Availability: {availability:.2f}%")
        data_availability[step][f'{home}_{file}'] = availability

Home: H1, File: Indoor.csv, Data Availability: 55.47%
Home: H1, File: Outdoor.csv, Data Availability: 45.67%
Home: H2, File: Indoor.csv, Data Availability: 91.92%
Home: H2, File: Outdoor.csv, Data Availability: 92.74%
Home: H3, File: Indoor.csv, Data Availability: 99.06%
Home: H3, File: Outdoor.csv, Data Availability: 90.15%
Home: H4, File: Indoor.csv, Data Availability: 88.16%
Home: H4, File: Outdoor.csv, Data Availability: 95.99%
Home: H5, File: Indoor.csv, Data Availability: 77.19%
Home: H5, File: Outdoor.csv, Data Availability: 56.40%
Home: H6, File: Indoor.csv, Data Availability: 97.48%
Home: H6, File: Outdoor.csv, Data Availability: 95.44%


In [49]:
## Delete data if pm2.5 < 100 um/m3 and delta > 10 um/m3, or if pm2.5 > 100um/m3  and delta > 10%
import os
step = '4a'
data_availability[step] = {"Description" : "Delete data if pm2.5 < 100 um/m3 and delta > 10 um/m3, or if pm2.5 > 100um/m3  and delta > 10%"}

dir = 'merged'
homes = [f for f in os.listdir(dir) if os.path.isdir(os.path.join(dir, f))]
for home in homes:
    files = ['Indoor.csv', 'Outdoor.csv']
    for file in files:
        file_path = os.path.join(dir, home, file)
        df = pd.read_csv(file_path, parse_dates=['BDDateTime'])
        total_sec = (df['BDDateTime'].max() - df['BDDateTime'].min()).total_seconds()
        expected_data = total_sec / 120

        #logics
        pm_logic = ~df['pm2_5_atm'].isna() & ~df['pm2_5_atm_b'].isna()
        rh_logic = (df['current_humidity'] < 100) & (df['current_humidity'] > 0)
        temp_logic = (df['current_temp_f'] > -200) & (df['current_temp_f'] < 1000)
        delta = (df['pm2_5_atm'] - df['pm2_5_atm_b']).abs()
        average = (df['pm2_5_atm'] + df['pm2_5_atm_b'])/2
        cond1 = (delta <= 10) & (average <= 100)
        cond2 = (delta <= 0.1*average) & (average > 100)
        pm_channel_merge_logic = cond1 | cond2
        validity_logic = pm_logic & rh_logic & temp_logic & pm_channel_merge_logic

        #avialability check
        available_data = df[validity_logic].shape[0]
        availability = (available_data / expected_data) * 100 if expected_data > 0 else 0
        print(f"Home: {home}, File: {file}, Data Availability: {availability:.2f}%")
        data_availability[step][f'{home}_{file}'] = availability

Home: H1, File: Indoor.csv, Data Availability: 53.02%
Home: H1, File: Outdoor.csv, Data Availability: 45.35%
Home: H2, File: Indoor.csv, Data Availability: 81.78%
Home: H2, File: Outdoor.csv, Data Availability: 65.25%
Home: H3, File: Indoor.csv, Data Availability: 72.98%
Home: H3, File: Outdoor.csv, Data Availability: 88.97%
Home: H4, File: Indoor.csv, Data Availability: 73.95%
Home: H4, File: Outdoor.csv, Data Availability: 81.14%
Home: H5, File: Indoor.csv, Data Availability: 48.07%
Home: H5, File: Outdoor.csv, Data Availability: 44.62%
Home: H6, File: Indoor.csv, Data Availability: 66.42%
Home: H6, File: Outdoor.csv, Data Availability: 77.52%


In [53]:
## Delete data delta > 5ug/m3 and delta > 61% a
import os
step = '4b'
data_availability[step] = {"Description" : "Delete data delta > 5ug/m3 and delta > 61% a"}

dir = 'merged'
homes = [f for f in os.listdir(dir) if os.path.isdir(os.path.join(dir, f))]
for home in homes:
    files = ['Indoor.csv', 'Outdoor.csv']
    for file in files:
        file_path = os.path.join(dir, home, file)
        df = pd.read_csv(file_path, parse_dates=['BDDateTime'])
        total_sec = (df['BDDateTime'].max() - df['BDDateTime'].min()).total_seconds()
        expected_data = total_sec / 120

        #logics
        pm_logic = ~df['pm2_5_atm'].isna() & ~df['pm2_5_atm_b'].isna()
        rh_logic = (df['current_humidity'] < 100) & (df['current_humidity'] > 0)
        temp_logic = (df['current_temp_f'] > -200) & (df['current_temp_f'] < 1000)
        delta = (df['pm2_5_atm'] - df['pm2_5_atm_b']).abs()
        average = (df['pm2_5_atm'] + df['pm2_5_atm_b'])/2
        cond1 = (delta <= 10) & (average <= 100)
        cond2 = (delta <= 0.1*average) & (average > 100)
        pm_channel_merge_logic = cond1 | cond2
        pm_channel_merge_logic2 = (delta < 5) | (delta < 0.61 * df['pm2_5_atm'])
        validity_logic = pm_logic & rh_logic & temp_logic & pm_channel_merge_logic & pm_channel_merge_logic2

        #avialability check
        available_data = df[validity_logic].shape[0]
        availability = (available_data / expected_data) * 100 if expected_data > 0 else 0
        print(f"Home: {home}, File: {file}, Data Availability: {availability:.2f}%")
        data_availability[step][f'{home}_{file}'] = availability

Home: H1, File: Indoor.csv, Data Availability: 52.82%
Home: H1, File: Outdoor.csv, Data Availability: 45.35%
Home: H2, File: Indoor.csv, Data Availability: 81.78%
Home: H2, File: Outdoor.csv, Data Availability: 65.21%
Home: H3, File: Indoor.csv, Data Availability: 72.73%
Home: H3, File: Outdoor.csv, Data Availability: 88.97%
Home: H4, File: Indoor.csv, Data Availability: 73.95%
Home: H4, File: Outdoor.csv, Data Availability: 81.14%
Home: H5, File: Indoor.csv, Data Availability: 48.05%
Home: H5, File: Outdoor.csv, Data Availability: 44.62%
Home: H6, File: Indoor.csv, Data Availability: 65.52%
Home: H6, File: Outdoor.csv, Data Availability: 72.53%


In [55]:
availability_df = pd.DataFrame.from_dict(data_availability, orient='index')

In [57]:
availability_df.to_csv("availability.csv")

## Table a.2

In [61]:
hourly_availability = {}

In [83]:
## Delete data delta > 5ug/m3 and delta > 61% a
import os
import numpy as np

dir = 'hourly_average'
homes = [f for f in os.listdir(dir) if os.path.isdir(os.path.join(dir, f))]
for home in homes:
    files = ['Indoor.csv', 'Outdoor.csv']
    for file in files:
        sensor = f'{home}_{file}'
        hourly_availability[sensor] = {}
        file_path = os.path.join(dir, home, file)
        df = pd.read_csv(file_path, parse_dates=['time'])
        total_sec = (df['time'].max() - df['time'].min()).total_seconds()
        expected_data = total_sec / 3600

        #avialability check
        available_data = df[df['pm2_5_atm_comb'].notna()].shape[0]
        availability = (available_data / expected_data) * 100 if expected_data > 0 else 0

        pm_2_5_mean = df['pm2_5_atm_comb'].mean()
        pm_2_5_sd = df['pm2_5_atm_comb'].std()
        pm_2_5_gm = np.exp(np.mean(np.log(df['pm2_5_atm_comb'].dropna().to_numpy())))
        a = df['pm2_5_atm_comb'].dropna().to_numpy()
        pm_2_5_iqr = np.percentile(a, 75) - np.percentile(a, 25)

        rh_mean = df['current_humidity'].mean()
        rh_sd = df['current_humidity'].std()

        temp_mean = df['current_temp_f'].mean()
        temp_sd = df['current_temp_f'].std()

        hourly_availability[sensor] = {
            'Number of hours of data' : expected_data,
            'PM mean' : pm_2_5_mean,
            'PM SD' : pm_2_5_sd,
            'PM GM' : pm_2_5_gm,
            'PM IQR' : pm_2_5_iqr,
            'RH mean' : rh_mean,
            'RH SD' : rh_sd,
            'Temp mean' : temp_mean,
            "Temp SD" : temp_sd,
            'Hourly data completeness' : availability
        }

        print(f"Home: {home}, File: {file}, Data Availability: {availability:.2f}%")
        data_availability[step][f'{home}_{file}'] = availability

Home: H1, File: Indoor.csv, Data Availability: 55.94%
Home: H1, File: Outdoor.csv, Data Availability: 45.64%
Home: H2, File: Indoor.csv, Data Availability: 93.44%
Home: H2, File: Outdoor.csv, Data Availability: 96.34%
Home: H3, File: Indoor.csv, Data Availability: 99.65%
Home: H3, File: Outdoor.csv, Data Availability: 91.06%


C:\Users\Dipankar Mitra\AppData\Local\Temp\ipykernel_30520\419681075.py:23: RuntimeWarning: divide by zero encountered in log
  pm_2_5_gm = np.exp(np.mean(np.log(df['pm2_5_atm_comb'].dropna().to_numpy())))


Home: H4, File: Indoor.csv, Data Availability: 89.25%
Home: H4, File: Outdoor.csv, Data Availability: 96.26%
Home: H5, File: Indoor.csv, Data Availability: 77.75%
Home: H5, File: Outdoor.csv, Data Availability: 56.74%
Home: H6, File: Indoor.csv, Data Availability: 97.93%
Home: H6, File: Outdoor.csv, Data Availability: 96.13%


In [84]:
hourly_availability_df = pd.DataFrame.from_dict(hourly_availability, orient='index')

In [86]:
hourly_availability_df.to_csv("hourly_availability.csv")